# Preparation Agent
This notebook is dedicated to the data extraction and basic preparation done in the notebook 'Preparation Agent'. We start by installing dependencies that we need.
As previously, Helper functions will be used. Available on GitHub.

In [1]:
import pandas as pd
import numpy as np
import os
dir = 'D:/Master BWL HU/3. Semester/Seminar Information Systems/Seminar-Information-Systems-main'
os.chdir(dir)

from helper_functions import Helper
from agents import Preparation_Agent
import pandas as pd

helper = Helper()

dbfile  = "D:/Master BWL HU/3. Semester/Seminar Information Systems/Seminar-Information-Systems-main/home-assistant_Chris_v3.db"

C:\Users\chris\anaconda3\envs\recomm\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


The helper functions will be used as in the previous works. Available at GitHub

### SQL Data Pulling

I removed the code for other databases as we will stick to the one called 'states'. In my own database I see states and attributes in different databases. 
<font color='red'>Check with Alona how she managed to put them together. And which version do we use? Did she send us updated data?
</font> 
For now databases will be in the same form as in the first Alona's file.

Note, initialization function contains also the function for SQL Pulling. <font color='red'> If you have any optimization ideas, please bring it in! </font>

In [2]:
class Preparation_Agent:
    import pandas as pd
    helper = Helper()
    def __init__(self, dbfile, shiftable_devices):
        self.input = helper.export_sql(dbfile)
        self.shiftable_devices = shiftable_devices

In [3]:
shiftable_devices = ["sensor.shellyplug_s_4022d88961b4_power", "sensor.shellyplug_s_4022d88984b8_power"]

In [4]:
prep = Preparation_Agent(dbfile, shiftable_devices)
prep.input.head()

,state_id,entity_id,state,attributes,event_id,last_changed,last_updated,old_state_id,attributes_id,context_id,context_user_id,context_parent_id,origin_idx,hash,shared_attributes
0,1,update.home_assistant_supervisor_update,off,None,None,None,2022-12-25 18:18:34.594923,NaN,1,01GN5842Q23HBMB7EZ6KY9A13R,None,None,0,2939789592,"{""auto_update"":true,""installed_version"":""2022...."
1,2,update.home_assistant_core_update,off,None,None,None,2022-12-25 18:18:34.595605,NaN,2,01GN5842Q3P07WA0KC0S4WMRZ4,None,None,0,2651871198,"{""auto_update"":false,""installed_version"":""2022..."
2,3,update.home_assistant_operating_system_update,off,None,None,None,2022-12-25 18:18:34.596293,NaN,3,01GN5842Q4738MTF23J7WJFNA8,None,None,0,2287136131,"{""auto_update"":false,""installed_version"":""9.4""..."
3,4,sun.sun,below_horizon,None,None,None,2022-12-25 18:18:34.969050,NaN,4,01GN58432SXZ8BV6NEGP0NDMQF,None,None,0,1454287034,"{""friendly_name"":""Sun""}"
4,5,zone.home,0,None,None,None,2022-12-25 18:18:34.971450,NaN,5,01GN58432VZ0634BF3S40SSSPA,None,None,0,1858961947,"{""latitude"":0,""longitude"":0,""radius"":100,""pass..."


In [5]:
prep.input["shared_attributes"][1]

'{"auto_update":false,"installed_version":"2022.12.8","latest_version":"2022.12.8","release_url":"https://www.home-assistant.io/latest-release-notes/","skipped_version":null,"title":"Home Assistant Core","friendly_name":"Home Assistant Core Update"}'

### Data Unpacking
We received data frame that has dictionaries in column value. In this section, we bring the data frame to the wide format, additionally unpacking the column. After that I specify that I need values in kWh and choose nodes. <font color='red'>Note: user should specify the nodes. How does he do it? Is kWh okay?
</font>

Here I am dropping values but it is not sustainable solution.

In [6]:
#adapted for HA
def unpacking_attributes(self, df):
    output = df.copy()
    output['shared_attributes']=output['shared_attributes'].apply(lambda x: x.replace('true','True'))
    output['shared_attributes']=output['shared_attributes'].apply(lambda x: x.replace('false','False'))
    output['shared_attributes']=output['shared_attributes'].apply(lambda x: x.replace('null','None'))

    output['shared_attributes']=output['shared_attributes'].apply(lambda dat: dict(eval(dat)))
    df2 = pd.json_normalize(output['shared_attributes'])
    result = pd.DataFrame( pd.concat([output,df2], axis = 1).drop('shared_attributes', axis = 1))
    result = result.dropna(axis = 1, thresh=int(0.95*(len(result.columns))))
    return result

setattr(Preparation_Agent, 'unpacking_attributes', unpacking_attributes)
del unpacking_attributes

In [7]:
data_unpacked  = prep.unpacking_attributes(prep.input)

In [8]:
data_unpacked

,state_id,entity_id,state,last_changed,last_updated,old_state_id,attributes_id,context_id,origin_idx,hash,...,popularity_thursday,popularity_friday,popularity_saturday,popularity_sunday,gps_accuracy,altitude,course,vertical_accuracy,source,visibility
0,1,update.home_assistant_supervisor_update,off,None,2022-12-25 18:18:34.594923,NaN,1,01GN5842Q23HBMB7EZ6KY9A13R,0,2939789592,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,update.home_assistant_core_update,off,None,2022-12-25 18:18:34.595605,NaN,2,01GN5842Q3P07WA0KC0S4WMRZ4,0,2651871198,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,update.home_assistant_operating_system_update,off,None,2022-12-25 18:18:34.596293,NaN,3,01GN5842Q4738MTF23J7WJFNA8,0,2287136131,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,sun.sun,below_horizon,None,2022-12-25 18:18:34.969050,NaN,4,01GN58432SXZ8BV6NEGP0NDMQF,0,1454287034,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,zone.home,0,None,2022-12-25 18:18:34.971450,NaN,5,01GN58432VZ0634BF3S40SSSPA,0,1858961947,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48052,48053,sensor.bogenleuchte,3.64,None,2023-01-09 16:04:03.304076,48047.0,3187,01GPBMCHK8NKZ9BDN58J2R4VR7,0,119982377,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48053,48054,sensor.olaf,3.02,None,2023-01-09 16:04:04.365858,48048.0,3190,01GPBMCJMDFRDAMRBF5VH03ZH2,0,1243904514,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48054,48055,sensor.blub,3.02,None,2023-01-09 16:04:07.163404,48049.0,3189,01GPBMCNBVQG0WZZBYZK2FPCRN,0,1905492863,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48055,48056,sensor.bogenleuchte,3.66,None,2023-01-09 16:05:03.304884,48053.0,3187,01GPBMEC68DB8P98M3AQ9KNE4B,0,119982377,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Attributes ID is not unique for one entity, therefore we use entity_id to identify the different sensors

In [ ]:
data_unpacked.loc[data_unpacked['entity_id'] == "sensor.shellyplug_s_4022d88984b8_power"].attributes_id.unique()

In [ ]:
data_unpacked.unit_of_measurement.unique()

In [ ]:
W_data = data_unpacked[data_unpacked.unit_of_measurement=='W']
W_data

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(W_data[['last_updated', 'state']])

In [ ]:
W_data.columns

In [ ]:
W_data = W_data.iloc[:,0:13]
W_data

In [ ]:
W_data.entity_id.unique()

### Accessing only nodes in W
Not optimally written code! However, this function only takes entities that have consumption in kWh and nodes defined by the user. <font color = 'red'> Next question is how to merge it with the home assistant so user can define which node to use? Also tried to get rid of NaN column, why it didn't work out? </font>

In [ ]:
#updated
def access_shiftable_devices(self, df, attrs= 'all'):
    trial = df.copy()
    trial.attributes_id = trial.attributes_id.dropna()
    trial.state= pd.to_numeric(trial['state'], errors='coerce').dropna()
    if attrs == 'all':
        w_data = trial[trial.unit_of_measurement.isin(['W'])]
        w_data = trial[trial.entity_id.isin(prep.shiftable_devices)]
        #we can add entity_id, or disregard
        w_data_long = w_data[['entity_id','last_updated','state']]
        w_data_wide = pd.pivot(w_data_long,  index = ['last_updated'], columns = 'entity_id', values = 'state')
    if attrs != 'all':
        w_data = trial[trial.unit_of_measurement.isin(['W']) & trial.attributes_id.isin([attrs])]
        w_data = trial[trial.entity_id.isin(prep.shiftable_devices)]
        w_data_long = w_data[['entity_id','last_updated','state']]
        w_data_wide = pd.pivot(w_data_long,  index = ['last_updated'], columns = 'entity_id', values = 'state')
    result = w_data_wide.fillna(0).reset_index()
    return(result)

setattr(Preparation_Agent, 'access_shiftable_devices', access_shiftable_devices)
del access_shiftable_devices

In [ ]:
data_w = prep.access_shiftable_devices(data_unpacked)
data_w.head()

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(data_w[['last_updated','sensor.shellyplug_s_4022d88984b8_power']])

### Standard Data Preprocessing - Outlier Truncation
Here comes <font color='red'> the first issue </font>, the consumption which was reported is really low. When we apply outlier truncation, all values that are not zero becoming zeros. Therefore, should we skip outlier truncation step?
I am taking kWh - kilowatt hour. <font color='red'> Do you think I should take W better  </font>? Below one can see that only node 3 and 46 are active and activity is super low. I will create outlier truncation but I do not think that it should be applied.

In [ ]:
# Histogram
data_w.hist(bins=10, figsize=(8,8));

In [ ]:
data_w.isna().any()

Skewed data, no outliers <font color = 'red'> (as I personally believe) </font>. Data has no missing values. However, as I am not sure whether this will be the case with any data I will keep truncation functions.

In [ ]:
data_w.describe()

In [ ]:
data_w

In [ ]:
def outlier_truncation(self, series, factor=1.5, verbose=0):
    from tqdm import tqdm
    
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3-q1
    
    lower_bound = q1 - factor*iqr
    upper_bound = q3 + factor*iqr
    
    output = []
    counter = 0
    for item in (tqdm(series, desc=f'[outlier truncation: {series.name}]') if verbose != 0 else series):
        if item > upper_bound:
            output.append(int(upper_bound))
            counter += 1
        elif item < lower_bound:
            output.append(int(lower_bound))
            counter += 1
        else:
            output.append(item)
    print(f'[outlier truncation: {series.name}]: {counter} outliers were truncated.') if verbose != 0 else None 
    return output

# add the function to the class and delete global function
setattr(Preparation_Agent, 'outlier_truncation', outlier_truncation)
del outlier_truncation

In [ ]:
new_df = pd.DataFrame()
new_df['573 truncated'] = prep.outlier_truncation(data_w["sensor.shellyplug_s_4022d88984b8_power"], verbose = 1)
new_df.describe()

In [ ]:
# truncating multiple features
def truncate(self, df, features='all', factor=1.5, verbose=0):
    import time
    output = df.copy()
    features = df.select_dtypes(include=['int', 'float']).columns if features == 'all' else features

    for feature in features:
        time.sleep(0.2) if verbose != 0 else None
        row_nn = df[feature] != 0                                                                  # truncate only the values for which the device uses energy
        output.loc[row_nn, feature] = self.outlier_truncation(df.loc[row_nn, feature], factor=factor, verbose=verbose) # Truncatation factor = 1.5 * IQR
        print('\n') if verbose != 0 else None
    return output

# add to class
setattr(Preparation_Agent, 'truncate', truncate)
del truncate

In [ ]:
household_t = prep.truncate(data_w, verbose = 1)
household_t.describe().round(2)

### MinMax Scaling

In [ ]:
def scale(self, df, features='all', kind='MinMax', verbose=0):
    output = df.copy()
    features = df.select_dtypes(include=['int', 'float']).columns if features == 'all' else features
    
    if kind == 'MinMax':
        from sklearn.preprocessing import MinMaxScaler
        
        scaler = MinMaxScaler()
        output[features] = scaler.fit_transform(df[features])
        print('[MinMaxScaler] Finished scaling the data.') if verbose != 0 else None
    else:
        raise InputError('Chosen scaling method is not available.')
    return output 

# add to the class
setattr(Preparation_Agent, 'scale', scale)
del scale

In [ ]:
household_s = prep.scale(household_t, verbose = 0)

In [ ]:
household_s

### Feature Creation

**Resampling by hour**
First, I will create the function that aggregates the dataframe for the period X, it can be 60 minutes, it can be one day. It is resampled this way, that we receive sum of the variables per hour/day and not the average as in initial preparation agent. <font color = 'red'> If you consider that we should take mean, please let me know. </font>


In [ ]:
def get_timespan(self, df, start, timedelta_params):
    df.last_updated = pd.to_datetime(df.last_updated)
    df = df.set_index('last_updated')
    start = pd.to_datetime(start) if type(start) != type(pd.to_datetime('1970-01-01')) else start 
    end = start + pd.Timedelta(**timedelta_params)
    return df[start:end].reset_index()

# add to the helper class
setattr(Helper, 'get_timespan', get_timespan)
del get_timespan

In [ ]:
# Extracting data for one day
helper = Helper()
from datetime import datetime

timedelta_params = {'days': 1, 'seconds': -1}   # returns data for just one day
day = helper.get_timespan(data_w, "2023-01-01", timedelta_params)
day

In [ ]:
#aggregating per hour
data = day.resample('60T', on = 'last_updated').mean()
data.describe().round(3)


In [ ]:
data

<font color = 'red'> Mean gives back NaN if there are only 0! So we have to replace all nan values with 0. Otherwise later on the load agent does not find right load patterns.</font>.

In [ ]:
output = data_w.copy()
output.last_updated

In [ ]:
output['last_updated'] = pd.to_datetime(output['last_updated'])
output

In [ ]:
output.last_updated

In [ ]:
output = output.resample('60T', on="last_updated").mean()
output

In [ ]:
output = output.replace(np.nan, 0)
output

<font color = 'red'> In Zharova's approach there is function aggregate, I changed it to resampling. Note, she takes **mean**, I take sum as hourly total consumption </font>.

In [ ]:
#def aggregate(self, df, resample_param):
#    return df.resample(resample_param).mean().copy()

In [ ]:
# add to helper and not to preparation agent
def aggregate(self, df, period = '60T'):
    output = df.copy()
    output['last_updated'] = pd.to_datetime(output['last_updated'])
    output = output.resample(period, on="last_updated").mean()
    output = output.replace(np.nan, 0)
    return output

setattr(Preparation_Agent, 'aggregate', aggregate)
del aggregate

In [ ]:
data_w

In [ ]:
data = prep.aggregate(data_w)
data

In [ ]:
def plot_consumption(self, df, features='all', figsize='default', threshold=None, title='Consumption'):
    import matplotlib.pyplot as plt

    df = df.copy()
    features = [column for column in df.columns if column not in ['Unix', 'Issues']] if features == 'all' else features

    fig, ax = plt.subplots(figsize=figsize) if figsize != 'default' else plt.subplots()
    if threshold != None:
        df['threshold'] = [threshold]*df.shape[0]
        ax.plot(df['threshold'], color = 'tab:red')
    for feature in features:
        ax.plot(df[feature])
    ax.legend(['threshold'] + features) if threshold != None else ax.legend(features)
    ax.set_title(title);

# add to the class
setattr(Preparation_Agent, 'plot_consumption', plot_consumption)
del plot_consumption


<font color='red'> **How do we define the threshold in our case?** </font> I randomly took threshold in this case.


In [ ]:
prep.plot_consumption(data, threshold = 1)

**Device Usage Feature**

In [ ]:
def get_device_usage(self, df, device, threshold):
    return (df.loc[:, device] > threshold).astype('int')

# add to class
setattr(Preparation_Agent, 'get_device_usage', get_device_usage)
del get_device_usage

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(prep.get_device_usage(data, "sensor.shellyplug_s_4022d88984b8_power", 1))



In [ ]:
prep.get_device_usage(data, "sensor.shellyplug_s_4022d88961b4_power", 1)

Function that creates binary variable and aggregates in order to show whether variable has been or not used in particular day.

In [ ]:
## daily appliance usage
data_all = prep.aggregate(household_s, '1D')
shiftable_devices = ["sensor.shellyplug_s_4022d88961b4_power", "sensor.shellyplug_s_4022d88984b8_power"]

for appliance in shiftable_devices:
    data_all['node'+ str(appliance) + '_usage'] = prep.get_device_usage(data_all, appliance, 0)
data_all.filter(regex = 'usage')

Activity feature appear in my notebook a bit ealier than in original one.

In [ ]:
def get_activity(self, df, active_appliances, threshold):
    import pandas as pd
    active = pd.DataFrame({appliance: df[appliance] > threshold for appliance in active_appliances})
    return active.apply(any, axis = 1).astype('int')

# add to the Preparation_agent class
setattr(Preparation_Agent, 'get_activity', get_activity)
del get_activity

In [ ]:
shiftable_devices = ["sensor.shellyplug_s_4022d88961b4_power", "sensor.shellyplug_s_4022d88984b8_power"]

data_all['activity'] = prep.get_activity(data_all, shiftable_devices, 0)
data_all

**Periods since the last device usage**

In [ ]:
def get_last_usage(self, series):
    last_usage = []
    for idx in range(len(series)):
        shift = 1
        if pd.isna(series.shift(periods = 1)[idx]):
            shift = None
        else:
            while series.shift(periods = shift)[idx] == 0:
                shift += 1
        last_usage.append(shift)
    return last_usage

def get_last_usages(self, df, features):
    import pandas as pd

    output = pd.DataFrame()
    for feature in features:
        output['periods_since_last_'+str(feature)] = self.get_last_usage(df[feature])
    output.set_index(df.index, inplace=True)
    return output


setattr(Preparation_Agent, 'get_last_usage', get_last_usage)
del get_last_usage

setattr(Preparation_Agent, 'get_last_usages', get_last_usages)
del get_last_usages

In [ ]:
usage_cols = data_all.filter(regex = 'usage')

data_all = data_all.join(prep.get_last_usages(data_all, usage_cols))
data_all.iloc[:, -10:]

**Time Features Based on DatetimeIndex**

In [ ]:
def get_time_feature(self, df, features='all'):
    import pandas as pd
    functions = {
        'hour': lambda df: df.index.hour, 
        'day_of_week': lambda df: df.index.dayofweek,
        'day_name': lambda df: df.index.day_name().astype('category'),
        'month': lambda df: df.index.month, 
        'month_name': lambda df: df.index.month_name().astype('category'),
        'weekend': lambda df: [int(x in ['Saturday', 'Sunday']) for x in  list(df.index.day_name())]
    }
    if features == 'all':
        output = pd.DataFrame({function[0]: function[1](df) for function in functions.items()})
    else:
        output = pd.DataFrame({function[0]: function[1](df) for function in functions.items() if function[0] in features})
    output.set_index(df.index, inplace=True)
    return output

# add to the Preparation_agent class
setattr(Preparation_Agent, 'get_time_feature', get_time_feature)
del get_time_feature

In [ ]:
day

In [ ]:
day = day.set_index('last_updated')
day = day.loc[:,["sensor.shellyplug_s_4022d88961b4_power", "sensor.shellyplug_s_4022d88984b8_power"]]
day = day.reset_index()
day['last_updated']=pd.to_datetime(day['last_updated'])
day = day.set_index('last_updated')
day = day.join(prep.get_time_feature(day))

In [ ]:
day.head()

**Time Lag Features**

In [ ]:
def get_time_lags(self, df, features, lags):
    import pandas as pd

    output = pd.DataFrame()
    for feature in features:
        for lag in lags:
            output[f'{feature}_lag_{lag}'] = df[feature].shift(periods=lag)
    return output

# add to the Preparation_agent class
setattr(Preparation_Agent, 'get_time_lags', get_time_lags)
del get_time_lags

In [ ]:
day['activity'] = prep.get_activity(day, shiftable_devices, 1)

In [ ]:
day

In [ ]:
day = day.join(prep.get_time_lags(day, ['hour','activity'], [1, 2, 5]))
day.iloc[:, -6:]

### Defining optimal threshold


In [ ]:
helper.plot_consumption(day, shiftable_devices, figsize=(18,10), threshold=.15)

In [ ]:
def visualize_threshold(self, df, threshold, appliances, figsize=(18,5)):
    # data prep
    for appliance in appliances:
        df[str(appliance) + '_usage'] = self.get_device_usage(df, appliance, threshold)
    df = df.join(self.get_time_feature(df))
    df['activity'] = self.get_activity(df, appliances, threshold)

    # plotting 
    import matplotlib.pyplot as plt

    usage_cols = [column for column in df.columns if str(column).endswith('_usage')]
    columns = ['activity'] + usage_cols

    fig, axes = plt.subplots(1,3, figsize=figsize)

    # hour
    hour = df.groupby('hour').mean()[columns]
    hour.plot(ax=axes[0])
    axes[0].set_ylim(-.1, 1.1);
    axes[0].set_title(f'[threshold: {round(threshold, 4)}] Activity ratio per hour')

    # week 
    usage_cols = [column for column in df.columns if str(column).endswith('_usage')]
    week = df.groupby('day_name').mean()[columns]
    week = week.reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
    week.plot(ax=axes[1])
    axes[1].set_ylim(-.1, 1.1);
    #axes[1].set_xticklabels(['']+list(week.index), rotation=90)
    axes[1].set_title(f'[threshold: {round(threshold, 4)}] Activity ratio per day of the week')

    # month
    usage_cols = [column for column in df.columns if str(column).endswith('_usage')]
    month = df.groupby('month').mean()[columns]
    month.plot(ax=axes[2])
    axes[2].set_ylim(-.1, 1.1);
    axes[2].set_title(f'[threshold: {round(threshold, 4)}] Activity ratio per month')

# add to the Preparation_agent class
setattr(Preparation_Agent, 'visualize_threshold', visualize_threshold)
del visualize_threshold

In [ ]:
def validate_thresholds(self, df, thresholds, appliances, figsize=(18,5)):
    import time
    from tqdm import tqdm
    for threshold in tqdm(thresholds):
        self.visualize_threshold(df, threshold, appliances, figsize)
    time.sleep(0.2)
    print('\n')

# add to the Preparation_agent class
setattr(Preparation_Agent, 'validate_thresholds', validate_thresholds)
del validate_thresholds

In [ ]:
household_s = household_s.set_index('last_updated')
df = helper.aggregate(household_s, '7T')
thresholds = [0] + list(np.geomspace(.1, .5, 5))

prep.validate_thresholds(df, thresholds, shiftable_devices)

In [ ]:
thresholds

In [ ]:
threshold = .15

prep.visualize_threshold(df, threshold, shiftable_devices)

### Create dummy test data

In [ ]:
def last_reported(self, df):
    return str(df.index.max())[:10]

# add to the Preparation_agent class
setattr(Preparation_Agent, 'last_reported', last_reported)
del last_reported

In [ ]:
def add_dummy_data_tomorrow(self, df):
    date = self.last_reported(df)
    tomorrow = (pd.to_datetime(date) + pd.Timedelta(days=1)).strftime("%Y-%m-%d")
    
    # determine how many hours we need to fill up (missing hours till new day + 23 for tomorrow, day of prediction)
    hours_to_fill = 24 - int(str(df.index.max())[11:13]) + 23
    
    # add rows and fill up with dummy 0
    for i in range(0,hours_to_fill):
        idx = df.tail(1).index[0] + pd.Timedelta(hours=1)
        df.loc[idx] = 0
    return df
# add to the Preparation_agent class
setattr(Preparation_Agent, 'add_dummy_data_tomorrow', add_dummy_data_tomorrow)
del add_dummy_data_tomorrow

In [ ]:
prep = Preparation_Agent(dbfile, shiftable_devices)

In [ ]:
df  = prep.unpacking_attributes(helper.export_sql(dbfile))
df = prep.access_shiftable_devices(df)
df['last_updated'] = pd.to_datetime(df['last_updated'])
df = df.set_index('last_updated')
# Aggregate to hour level
df = helper.aggregate_load(df, '60T')

In [ ]:
df

In [ ]:
df = prep.add_dummy_data_tomorrow(df)
df

### Activity Pipeline


In [ ]:
truncation_params = {
    'features': 'all', 
    'factor': 1.5, 
    'verbose': 1
}

scale_params = {
    'features': 'all', 
    'kind': 'MinMax', 
    'verbose': 1
}

aggregate_params = {
    'resample_param': '60T'
}
#update with active appliances attributes_ids
activity_params = {
    'active_appliances': shiftable_devices,
    'threshold': .15
}

time_params = {
    'features': ['hour', 'day_name']
}

activity_lag_params = {
    'features': ['activity'],
    'lags': [24, 48, 72]
}

activity_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'aggregate': aggregate_params,
    'activity': activity_params,
    'time': time_params,
    'activity_lag': activity_lag_params
}

In [ ]:
#pipeline activity
def pipeline_activity(self, df, params):
    from helper_functions import Helper
    import pandas as pd
    helper = Helper()
    df = df.copy()
    import pandas as pd
    output = pd.DataFrame()

    df  = self.unpacking_attributes(self.input)
    df = self.access_shiftable_devices(df)
    # Data cleaning
    # df = self.truncate(df, **params['truncate'],)
    # df = self.scale(df, **params['scale'])
    # ignore scaling for now, we would just scale those variables, which does not make sense 
    # Index(['state_id', 'old_state_id', 'attributes_id', 'origin_idx', 'hash'], dtype='object')

    df['last_updated'] = pd.to_datetime(df['last_updated'])
    df = df.set_index('last_updated')
    # Aggregate to hour level
    df = helper.aggregate_load(df, **params['aggregate'])
    
    # Add dummy data
    df = self.add_dummy_data_tomorrow(df)

    # Activity feature
    output['activity'] = self.get_activity(df, **params['activity'])

    ## Time feature
    output = output.join(self.get_time_feature(df, **params['time']))

    # Activity lags
    output = output.join(self.get_time_lags(output, **params['activity_lag']))

    # Dummy coding
    output = pd.get_dummies(output, drop_first=True)

    return output

# add to the Preparation_agent class
setattr(Preparation_Agent, 'pipeline_activity', pipeline_activity)
del pipeline_activity

In [ ]:
prep = Preparation_Agent(dbfile, shiftable_devices)

In [ ]:
prep.pipeline_activity(prep.input, activity_pipe_params)

### Load Pipeline

The load agent requires cleaned energy consumption data for the shiftable devices. Therefore, we calculate the device usage for these appliances based on the truncated and scaled data and return the energy consumption of the shiftable devices only if we detected a device usage. The data for the load agent will be aggregated to the hour level.


In [ ]:
truncation_params = {
    'features': 'all', 
    'factor': 1.5, 
    'verbose': 1
}

scale_params = {
    'features': 'all', 
    'kind': 'MinMax', 
    'verbose': 1
}

aggregate_params = {
    'resample_param': '60T'
}

shiftable_devices = ["sensor.shellyplug_s_4022d88961b4_power", "sensor.shellyplug_s_4022d88984b8_power"]

device_params = {
    'threshold': 0.15
}

load_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'aggregate': aggregate_params,
    'shiftable_devices': shiftable_devices, 
    'device': device_params
}

In [ ]:
#pipeline load
def pipeline_load(self, df, params):
    from helper_functions import Helper
    import pandas as pd
    helper = Helper()

    df  = self.unpacking_attributes(self.input)
    df = self.access_shiftable_devices(df)

    df = df.copy()
    output = pd.DataFrame()

    # Data cleaning
    # df = self.truncate(df, **params['truncate'],)
    # scaled = self.scale(df, **params['scale'])
    # ignore scaling for now, we would just scale those variables, which does not make sense 
    # Index(['state_id', 'old_state_id', 'attributes_id', 'origin_idx', 'hash'], dtype='object')
    scaled = df.copy()

    df['last_updated'] = pd.to_datetime(df['last_updated'])
    df = df.set_index('last_updated')

    scaled['last_updated'] = pd.to_datetime(scaled['last_updated'])
    scaled = scaled.set_index('last_updated')

    # aggregate
    df = helper.aggregate_load(df, **params['aggregate'])
    scaled = helper.aggregate_load(scaled, **params['aggregate'])
    
    # add dummy data
    df = self.add_dummy_data_tomorrow(df)

    # Get device usage and transform to energy consumption
    for device in params['shiftable_devices']:
        df[str(device) + '_usage'] = self.get_device_usage(df, device, **params['device'])
        output[device] = df.apply(lambda timestamp: timestamp[device] * timestamp[str(device) + '_usage'], axis = 1)

    return output, scaled, df

# add to the Preparation_agent class
setattr(Preparation_Agent, 'pipeline_load', pipeline_load)
del pipeline_load

In [ ]:
output, scaled, df = prep.pipeline_load(prep.input, load_pipe_params)
output

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df['sensor.shellyplug_s_4022d88984b8_power'])

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(output['sensor.shellyplug_s_4022d88984b8_power'])


<font color = "red"> Here, what should I do with NAs? </font>

### Usage Pipeline

We will use the same functionality from the activity pipeline to create the activity feature for the usage agent. Furthermore, we create the device usage features for the appliances for which we will provide recommendations. As appliances for which we will provide recommendations, we used appliances that require user interaction and are time-shiftable, i.e. not bound to any specific time during the day (e.g. Television, Computer, etc.). These appliances will be called “shiftable devices”. Additionally, we create the periods since last usage feature for the shiftable devices, create time lags for activity and the shiftable devices, add further time features and aggregate the data to the day level. 


In [ ]:
truncation_params = {
    'features': 'all', 
    'factor': 1.5, 
    'verbose': 1
}

scale_params = {
    'features': 'all', 
    'kind': 'MinMax', 
    'verbose': 1
}

aggregate_hour_params = {
    'resample_param': '60T'
}

activity_params = {
    'active_appliances': shiftable_devices,
    'threshold': .1
}

shiftable_devices = ["sensor.shellyplug_s_4022d88961b4_power", "sensor.shellyplug_s_4022d88984b8_power"]

device_params = {
    'threshold': 0.15
}

aggregate_day_params = {
    'resample_param': '1D'
}

time_params = {
    'features': ['day_name', 'weekend']
}

usage_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'aggregate_hour': aggregate_hour_params,
    'activity': activity_params,
    'aggregate_day': aggregate_day_params,
    'shiftable_devices': shiftable_devices, 
    'device': device_params,
    'time': time_params
}

In [ ]:
#pipeline usage
def pipeline_usage(self, df, params):
    from helper_functions import Helper
    import pandas as pd

    helper = Helper()

    df  = self.unpacking_attributes(self.input)
    df = self.access_shiftable_devices(df)

    df = df.copy()
    output = pd.DataFrame()

    # Data cleaning
    # df = self.truncate(df, **params['truncate'],)
    # scaled = self.scale(df, **params['scale'])
    # ignore scaling for now, we would just scale those variables, which does not make sense 
    # Index(['state_id', 'old_state_id', 'attributes_id', 'origin_idx', 'hash'], dtype='object')
    scaled = df.copy()

    # df['last_updated'] = pd.to_datetime(df['last_updated'])
    # df = df.set_index('last_updated')
    scaled['last_updated'] = pd.to_datetime(scaled['last_updated'])
    scaled = scaled.set_index('last_updated')

    # Aggregate to hour level
    scaled = helper.aggregate_load(scaled, **params['aggregate_hour'])
    
    # Add dummy data
    scaled = self.add_dummy_data_tomorrow(scaled)

    # Activity feature
    output['activity'] = self.get_activity(scaled, **params['activity'])

    # Get device usage and transform to energy consumption
    for device in params['shiftable_devices']:
        output[str(device) + '_usage'] = self.get_device_usage(scaled, device, **params['device'])

    # aggregate and convert from mean to binary
    output = helper.aggregate(output, **params['aggregate_day'])
    output = output.apply(lambda x: (x > 0).astype('int'))

    # Last usage
    output = output.join(self.get_last_usages(output, output.columns))

    # Time features
    output = output.join(self.get_time_feature(output, **params['time']))

    # lags
    output = output.join(self.get_time_lags(output, ['activity'] + [str(device)+'_usage' for device in params['shiftable_devices']], [1,2,3]))
    output['active_last_2_days'] = ((output.activity_lag_1 == 1) | (output.activity_lag_2 == 1)).astype('int')

    # dummy coding
    output = pd.get_dummies(output, drop_first=True)
    return output

# add to the Preparation_agent class
setattr(Preparation_Agent, 'pipeline_usage', pipeline_usage)
del pipeline_usage

In [ ]:
output = prep.pipeline_usage(prep.input, usage_pipe_params)

In [ ]:
output

### Final: Complete Preparation Agent Class

In [ ]:
###################################################################################################
# preparation agent ###############################################################################
###################################################################################################
class Preparation_Agent:
#installing dependencies
    import pandas as pd
    #uploading data and simple data wrangling
    def __init__(self, dbfile, shiftable_devices):
        from helper_functions import Helper
        helper = Helper()
        self.input = helper.export_sql(dbfile)
        self.shiftable_devices = shiftable_devices

    def unpacking_attributes(self, df):
        import pandas as pd
        output = df.copy()
        output['shared_attributes']=output['shared_attributes'].apply(lambda x: x.replace('true','True'))
        output['shared_attributes']=output['shared_attributes'].apply(lambda x: x.replace('false','False'))
        output['shared_attributes']=output['shared_attributes'].apply(lambda x: x.replace('null','None'))

        output['shared_attributes']=output['shared_attributes'].apply(lambda dat: dict(eval(dat)))
        df2 = pd.json_normalize(output['shared_attributes'])
        result = pd.DataFrame( pd.concat([output,df2], axis = 1).drop('shared_attributes', axis = 1))
        result = result.dropna(axis = 1, thresh=int(0.95*(len(result.columns))))
        return result

    def access_shiftable_devices(self, df, attrs= 'all'):
        import pandas as pd
        trial = df.copy()
        trial.attributes_id = trial.attributes_id.dropna()
        trial.state= pd.to_numeric(trial['state'], errors='coerce').dropna()
        if attrs == 'all':
            w_data = trial[trial.unit_of_measurement.isin(['W'])]
            w_data = trial[trial.entity_id.isin(self.shiftable_devices)]
            w_data_long = w_data[['entity_id','last_updated','state']]
            w_data_wide = pd.pivot(w_data_long,  index = ['last_updated'], columns = 'entity_id', values = 'state')
        if attrs != 'all':
            w_data = trial[trial.unit_of_measurement.isin(['W']) & trial.attributes_id.isin([attrs])]
            w_data = trial[trial.entity_id.isin(self.shiftable_devices)]
            w_data_long = w_data[['entity_id','last_updated','state']]
            w_data_wide = pd.pivot(w_data_long,  index = ['last_updated'], columns = 'entity_id', values = 'state')
        result = w_data_wide.fillna(0).reset_index()
        return(result)
    
    #basic preprocessing
    # -------------------------------------------------------------------------------------------
    def outlier_truncation(self, series, factor=1.5, verbose=0):
        from tqdm import tqdm
        
        q1 = series.quantile(0.25)
        q3 = series.quantile(0.75)
        iqr = q3-q1
        
        lower_bound = q1 - factor*iqr
        upper_bound = q3 + factor*iqr
        
        output = []
        counter = 0
        for item in (tqdm(series, desc=f'[outlier truncation: {series.name}]') if verbose != 0 else series):
            if item > upper_bound:
                output.append(int(upper_bound))
                counter += 1
            elif item < lower_bound:
                output.append(int(lower_bound))
                counter += 1
            else:
                output.append(item)
        print(f'[outlier truncation: {series.name}]: {counter} outliers were truncated.') if verbose != 0 else None 
        return output
    
    def scale(self, df, features='all', kind='MinMax', verbose=0):
        output = df.copy()
        features = output.select_dtypes(include=['int', 'float']).columns if features == 'all' else features

        if kind == 'MinMax':
            from sklearn.preprocessing import MinMaxScaler
            
            scaler = MinMaxScaler()
            output[features] = scaler.fit_transform(output[features])
            print('[MinMaxScaler] Finished scaling the data.') if verbose != 0 else None
        else:
            raise InputError('Chosen scaling method is not available.')
        return output 

    def get_timespan(self, df, start, timedelta_params):
        df.last_updated = pd.to_datetime(df.last_updated)
        df = df.set_index('last_updated')
        start = pd.to_datetime(start) if type(start) != type(pd.to_datetime('1970-01-01')) else start 
        end = start + pd.Timedelta(**timedelta_params)
        return df[start:end].reset_index()
    
    def truncate(self, df, features='all', factor=1.5, verbose=0):
        import time
        output = df.copy()
        features = df.select_dtypes(include=['int', 'float']).columns if features == 'all' else features

        for feature in features:
            time.sleep(0.2) if verbose != 0 else None
            row_nn = df[feature] != 0                                                                  # truncate only the values for which the device uses energy
            output.loc[row_nn, feature] = self.outlier_truncation(df.loc[row_nn, feature], factor=factor, verbose=verbose) # Truncatation factor = 1.5 * IQR
            print('\n') if verbose != 0 else None
        return output
    
    def last_reported(self, df):
        return str(df.index.max())[:10]
    
    def add_dummy_data_tomorrow(self, df):
        date = self.last_reported(df)
        tomorrow = (pd.to_datetime(date) + pd.Timedelta(days=1)).strftime("%Y-%m-%d")

        # determine how many hours we need to fill up (missing hours till new day + 23 for tomorrow, day of prediction)
        hours_to_fill = 24 - int(str(df.index.max())[11:13]) + 23

        # add rows and fill up with dummy 0
        for i in range(0,hours_to_fill):
            idx = df.tail(1).index[0] + pd.Timedelta(hours=1)
            df.loc[idx] = 0
        return df

    
    def plot_consumption(self, df, features='all', figsize='default', threshold=None, title='Consumption'):
        df = df.copy()
        features = [column for column in df.columns if column not in ['Unix', 'Issues']] if features == 'all' else features
        fig, ax = plt.subplots(figsize=figsize) if figsize != 'default' else plt.subplots()
        if threshold != None:
            df['threshold'] = [threshold]*df.shape[0]
            ax.plot(df['threshold'], color = 'tab:red')
        for feature in features:
            ax.plot(df[feature])
        ax.legend(['threshold'] + features) if threshold != None else ax.legend(features)
        ax.set_title(title);
    # feature creation
    # -------------------------------------------------------------------------------------------
    def get_device_usage(self, df, device, threshold):
        return (df.loc[:, device] > threshold).astype('int')

    def get_activity(self, df, active_appliances, threshold):
        import pandas as pd
        active = pd.DataFrame({appliance: df[appliance] > threshold for appliance in active_appliances})
        return active.apply(any, axis = 1).astype('int')

    def get_last_usage(self, series):
        import pandas as pd
        last_usage = []
        for idx in range(len(series)):
            shift = 1
            if pd.isna(series.shift(periods = 1)[idx]):
                shift = None
            else:
                while series.shift(periods = shift)[idx] == 0:
                    shift += 1
            last_usage.append(shift)
        return last_usage

    def get_last_usages(self, df, features):
        import pandas as pd

        output = pd.DataFrame()
        for feature in features:
            output['periods_since_last_'+str(feature)] = self.get_last_usage(df[feature])
        output.set_index(df.index, inplace=True)
        return output


    def get_time_feature(self, df, features='all'):
        import pandas as pd
        functions = {
            'hour': lambda df: df.index.hour, 
            'day_of_week': lambda df: df.index.dayofweek,
            'day_name': lambda df: df.index.day_name().astype('category'),
            'month': lambda df: df.index.month, 
            'month_name': lambda df: df.index.month_name().astype('category'),
            'weekend': lambda df: [int(x in ['Saturday', 'Sunday']) for x in  list(df.index.day_name())]
        }
        if features == 'all':
            output = pd.DataFrame({function[0]: function[1](df) for function in functions.items()})
        else:
            output = pd.DataFrame({function[0]: function[1](df) for function in functions.items() if function[0] in features})
        output.set_index(df.index, inplace=True)
        return output
    
    def get_time_lags(self, df, features, lags):
        import pandas as pd
        output = pd.DataFrame()
        for feature in features:
            for lag in lags:
                output[f'{feature}_lag_{lag}'] = df[feature].shift(periods=lag)
        return output

    #visualising threshold
    # ------------------------------------------------------------------------------------------- 
    def visualize_threshold(self, df, threshold, appliances, figsize=(18,5)):
        import pandas as pd
        # data prep
        for appliance in appliances:
            df[str(appliance) + '_usage'] = self.get_device_usage(df, appliance, threshold)
        df = df.join(self.get_time_feature(df))
        df['activity'] = self.get_activity(df, appliances, threshold)

        # plotting 
        import matplotlib.pyplot as plt

        usage_cols = [column for column in df.columns if str(column).endswith('_usage')]
        columns = ['activity'] + usage_cols

        fig, axes = plt.subplots(1,3, figsize=figsize)

        # hour
        hour = df.groupby('hour').mean()[columns]
        hour.plot(ax=axes[0])
        axes[0].set_ylim(-.1, 1.1);
        axes[0].set_title(f'[threshold: {round(threshold, 4)}] Activity ratio per hour')

        # week 
        usage_cols = [column for column in df.columns if str(column).endswith('_usage')]
        week = df.groupby('day_name').mean()[columns]
        week = week.reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
        week.plot(ax=axes[1])
        axes[1].set_ylim(-.1, 1.1);
        #axes[1].set_xticklabels(['']+list(week.index), rotation=90)
        axes[1].set_title(f'[threshold: {round(threshold, 4)}] Activity ratio per day of the week')

        # month
        usage_cols = [column for column in df.columns if str(column).endswith('_usage')]
        month = df.groupby('month').mean()[columns]
        month.plot(ax=axes[2])
        axes[2].set_ylim(-.1, 1.1);
        axes[2].set_title(f'[threshold: {round(threshold, 4)}] Activity ratio per month')
    def validate_thresholds(self, df, thresholds, appliances, figsize=(18,5)):

        for threshold in tqdm(thresholds):
            self.visualize_threshold(df, threshold, appliances, figsize)
        time.sleep(0.2)
        print('\n')
    
    #pipelines
    # -------------------------------------------------------------------------------------------
    #pipeline load
    def pipeline_load(self, df, params):
        from helper_functions import Helper
        import pandas as pd
        helper = Helper()
        
        df  = self.unpacking_attributes(self.input)
        df = self.access_shiftable_devices(df)
        
        df = df.copy()
        output = pd.DataFrame()

        # Data cleaning
        # df = self.truncate(df, **params['truncate'],)
        # scaled = self.scale(df, **params['scale'])
        # ignore scaling for now, we would just scale those variables, which does not make sense 
        # Index(['state_id', 'old_state_id', 'attributes_id', 'origin_idx', 'hash'], dtype='object')
        scaled = df.copy()

        df['last_updated'] = pd.to_datetime(df['last_updated'])
        df = df.set_index('last_updated')

        scaled['last_updated'] = pd.to_datetime(scaled['last_updated'])
        scaled = scaled.set_index('last_updated')

        # aggregate
        df = helper.aggregate_load(df, **params['aggregate'])
        scaled = helper.aggregate_load(scaled, **params['aggregate'])
        
        # Add dummy data
        df = self.add_dummy_data_tomorrow(df)
        scaled = self.add_dummy_data_tomorrow(scaled)

        # Get device usage and transform to energy consumption
        for device in params['shiftable_devices']:
            df[str(device) + '_usage'] = self.get_device_usage(df, device, **params['device'])
            output[device] = df.apply(lambda timestamp: timestamp[device] * timestamp[str(device) + '_usage'], axis = 1)

        return output, scaled, df
    #pipeline usage
    def pipeline_usage(self, df, params):
        from helper_functions import Helper
        import pandas as pd

        helper = Helper()

        df  = self.unpacking_attributes(self.input)
        df = self.access_shiftable_devices(df)
        
        df = df.copy()
        output = pd.DataFrame()

        # Data cleaning
        # df = self.truncate(df, **params['truncate'],)
        # scaled = self.scale(df, **params['scale'])
        # ignore scaling for now, we would just scale those variables, which does not make sense 
        # Index(['state_id', 'old_state_id', 'attributes_id', 'origin_idx', 'hash'], dtype='object')
        scaled = df.copy()
        
        # df['last_updated'] = pd.to_datetime(df['last_updated'])
        # df = df.set_index('last_updated')
        scaled['last_updated'] = pd.to_datetime(scaled['last_updated'])
        scaled = scaled.set_index('last_updated')
        
        # Aggregate to hour level
        scaled = helper.aggregate_load(scaled, **params['aggregate_hour'])
        
        # Add dummy data
        scaled = self.add_dummy_data_tomorrow(scaled)


        # Activity feature
        output['activity'] = self.get_activity(scaled, **params['activity'])

        # Get device usage and transform to energy consumption
        for device in params['shiftable_devices']:
            output[str(device) + '_usage'] = self.get_device_usage(scaled, device, **params['device'])

        # aggregate and convert from mean to binary
        output = helper.aggregate(output, **params['aggregate_day'])
        output = output.apply(lambda x: (x > 0).astype('int'))

        # Last usage
        output = output.join(self.get_last_usages(output, output.columns))
        
        # Time features
        output = output.join(self.get_time_feature(output, **params['time']))

        # lags
        output = output.join(self.get_time_lags(output, ['activity'] + [str(device)+'_usage' for device in params['shiftable_devices']], [1,2,3]))
        output['active_last_2_days'] = ((output.activity_lag_1 == 1) | (output.activity_lag_2 == 1)).astype('int')

        # dummy coding
        output = pd.get_dummies(output, drop_first=True)
        return output

    #pipeline activity
    def pipeline_activity(self, df, params):
        from helper_functions import Helper
        import pandas as pd
        helper = Helper()
        df = df.copy()
        import pandas as pd
        output = pd.DataFrame()

        df  = self.unpacking_attributes(self.input)
        df = self.access_shiftable_devices(df)
        # Data cleaning
        # df = self.truncate(df, **params['truncate'],)
        # df = self.scale(df, **params['scale'])
        # ignore scaling for now, we would just scale those variables, which does not make sense 
        # Index(['state_id', 'old_state_id', 'attributes_id', 'origin_idx', 'hash'], dtype='object')

        df['last_updated'] = pd.to_datetime(df['last_updated'])
        df = df.set_index('last_updated')
        # Aggregate to hour level
        df = helper.aggregate_load(df, **params['aggregate'])
        
        # Add dummy data
        df = self.add_dummy_data_tomorrow(df)

        # Activity feature
        output['activity'] = self.get_activity(df, **params['activity'])
        
        ## Time feature
        output = output.join(self.get_time_feature(df, **params['time']))

        # Activity lags
        output = output.join(self.get_time_lags(output, **params['activity_lag']))

        # Dummy coding
        output = pd.get_dummies(output, drop_first=True)

        return output


In [ ]:
prep = Preparation_Agent(helper.export_sql(dbfile), shiftable_devices)
df = prep.pipeline_activity(helper.export_sql(dbfile), activity_pipe_params)
df

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)